In [ ]:
import yanex.results as yr

exps = yr.get_experiments(status=["completed"])
print(len(exps))
exps

10


[Experiment(id='eedd8fe3', name='staged-test', status='completed'),
 Experiment(id='a4811aca', name='[unnamed]', status='completed'),
 Experiment(id='5c7fe763', name='second-sweep-batch_size_64', status='completed'),
 Experiment(id='fa944a1a', name='staged-test', status='completed'),
 Experiment(id='18bd35cd', name='second-sweep-batch_size_64', status='completed'),
 Experiment(id='3a88d928', name='second-sweep-batch_size_32', status='completed'),
 Experiment(id='f764a7dd', name='second-sweep-batch_size_16', status='completed'),
 Experiment(id='53a3bb66', name='first-sweep-lr_0p02', status='completed'),
 Experiment(id='578842c3', name='first-sweep-lr_0p01', status='completed'),
 Experiment(id='bbaeb197', name='sweep-lr_0p01-batch_size_32-epochs_20', status='completed')]